In [30]:
import urbans
from dictionary import dic_en2jp

In [31]:
"""
TODO: Terminals for participles
V VT VsrT VMod VsrMod VPassOrgT VPassOrgMod

""" 
src_grammar = """S -> NPQ VP 
S -> NPQ was NominalSub 
S -> NPQ did NPVMod 
S -> was NominalSub Vobl 
S -> NPQ Vobl 
S -> was NominalSub Adj 
S -> was NominalSub Nominal 
S -> did NominalSub VP 

NPVMod -> Nominal VMod
NPVMod -> Nominal VPrepOrgMod
NPVT -> Nominal VT
NPVT -> Nominal VPrepOrgT

VP -> V Nominal
VP -> was Vobl
VPT -> VT Nominal
VPT -> was VoblT 
VPMod -> VMod Nominal
VPMod -> was VoblMod

VPrep -> was VPrep
VPrep -> Vsr by
VPrepT -> was VPrepT
VPrepT -> VsrT by
VPrepMod -> was VPrepMod
VPrepMod -> VsrMod by

VPrepOrgMod -> was VPrepOrgMod
VPrepOrgMod -> VPassOrgMod by
VPrepOrgT -> was VPrepOrgT
VPrepOrgT -> VPassOrgT by

Vobl -> VPrep Nominal
VoblT -> VPrepT Nominal 
VoblMod -> VPrepMod Nominal

NPQ -> WhWNominal 
NPQ -> WhW role caseO

commonNoun -> commonNoun RC
WhWcommonNoun -> WhWcommonNoun RC
detSubcommonNoun -> detSubcommonNoun RC

WhWcommonNoun -> WhW commonNounModHead

RC -> VoblMod
RC -> R VPMod
RC -> R NPVMod
RC -> whose role VPMod

VP -> VPT andVP
VP -> VPx andVP
VPx -> VPT punctVPT 
VPx -> VPx punctVPT 
andVP -> conj VP 
andVP -> punct conj VP
punctVPT -> punct VPT

VPMod -> VPT andVPMod
VPMod -> VPx andVPMod
VPx -> VPT punctVPT 
VPx -> VPx punctVPT 
andVPMod -> conj VPMod
andVPMod -> punct conj VPMod
punctVPT -> punct VPT

Vobl -> VoblT andVobl
Vobl -> Voblx andVobl 
Voblx -> VoblT punctVoblT 
Voblx -> Voblx punctVoblT 
andVobl -> conj Vobl 
andVobl -> punct conj Vobl
punctVoblT -> punct VoblT

VoblMod -> VoblT andVoblMod
VoblMod -> Voblx andVoblMod 
Voblx -> VoblT punctVoblT 
Voblx -> Voblx punctVoblT 
andVoblMod -> conj VoblMod 
andVoblMod -> punct conj VoblMod
punctVoblT -> punct VoblT

VPrep -> VPrepT andVPrep 
VPrep -> VPrepX andVPrep 
VPrepX -> VPrepT punctVPrepT 
VPrepX -> VPrepX punctVPrepT 
andVPrep -> conj VPrep 
andVPrep -> punct conj VPrep 
punctVPrepT -> punct VPrepT

VPrepMod -> VPrepT andVPrepMod 
VPrepMod -> VPrepX andVPrepMod 
VPrepX -> VPrepT punctVPrepT 
VPrepX -> VPrepX punctVPrepT 
andVPrepMod -> conj VPrepMod 
andVPrepMod -> punct conj VPrepMod 
punctVPrepT -> punct VPrepT

VPrepOrgMod -> VPrepOrgT andVPrepOrgMod 
VPrepOrgMod -> VPrepOrgX andVPrepOrgMod 
VPrepOrgX -> VPrepOrgT punctVPrepOrgT 
VPrepOrgX -> VPrepOrgX punctVPrepOrgT 
andVPrepOrgMod -> conj VPrepOrgMod 
andVPrepOrgMod -> punct conj VPrepOrgMod 
punctVPrepOrgT -> punct VPrepOrgT

V -> VT andV
V -> Vx andV
Vx -> VT punctVT
Vx -> Vx punctVT 
andV -> conj V
andV -> punct conj V 
punctVT -> punct VT

VT -> VT andVT
VT -> Vx andVT
Vx -> VT punctVT
Vx -> Vx punctVT 
andVT -> conj VT
andVT -> punct conj VT 
punctVT -> punct VT

VMod -> VT andVMod
VMod -> Vx andVMod
andVMod -> conj VMod
andVMod -> punct conj VMod

Vsr -> VsrT andVsr
Vsr -> Vsrx andVsr
Vsrx -> VsrT punctVsrT
Vsrx -> Vsrx punctVsrT 
andVsr -> conj Vsr
andVsr -> punct conj Vsr 
punctVsrT -> punct VsrT

VsrT -> VsrT andVsrT
VsrT -> Vsrx andVsrT
andVsrT -> conj VsrT
andVsrT -> punct conj VsrT 
punctVsrT -> punct VsrT

VsrMod -> VsrT andVsrMod
VsrMod -> Vsrx andVsrMod
andVsrMod -> conj VsrMod
andVsrMod -> punct conj VsrMod

Vx -> Vx punctVPrepT
Vx -> VT punctVPrepT
V -> Vx andVPrep
V -> VT andVPrep
VPrep -> VPrepT andV
VPrep -> VPrepX andV 
VPrepX -> VPrepT punctVT 
VPrepX -> VPrepX punctVT


VPrepOrgMod -> VPrepOrgT andVOrgPassMod
VPrepOrgMod -> VPrepOrgX andVOrgPassMod
VPrepOrgX -> VPrepOrgT punctVOrgPassT
VPrepOrgX -> VPrepOrgX punctVOrgPassT

punctVOrgPassT -> punct VOrgPassT
andVOrgPassMod -> conj VOrgPassMod
andVOrgPassMod -> punct conj VOrgPassMod


VPassOrgMod -> VPassOrgT andVPassOrgMod
VPassOrgMod -> VPassOrgx andVPassOrgMod
VPassOrgx -> VPassOrgT punctVPassOrgT
VPassOrgx -> VPassOrgx punctVPassOrgT 
andVPassOrgMod -> conj VPassOrgMod
andVPassOrgMod -> punct conj VPassOrgMod 
punctVPassOrgT -> punct VPassOrgT


VMod -> Vx andVPrepMod
VMod -> VT andVPrepMod
VPrepMod -> VPrepT andVMod
VPrepMod -> VPrepX andVMod

NPVMod -> NPVT andNPVMod
NPVMod -> NPVx andNPVMod 
NPVx -> NPVT punctNPVT
NPVx -> NPVx punctNPVT 
andNPVMod -> conj NPVMod 
andNPVMod -> punct conj NPVMod 
punctNPVT -> punct NPVT

V -> F V
VT -> F VT
VMod -> F VMod
Vsr -> F Vsr
VsrT -> F VsrT
VsrMod -> F VsrMod
VPassOrgMod -> F VPassOrgMod
VPassOrgT -> F VPassOrgT
VOrgPassT -> F VOrgPassT
VOrgPassMod -> F VOrgPassMod


Nominal -> Name 
Nominal -> DP
Nominal -> commonNoun
WhWNominal -> WhWcommonNoun

NominalSub -> Name
NominalSub -> DPSub
NominalSub -> commonNoun

DP -> caseS role
DPs -> caseS role
DPSub -> caseSSub role
DPSubs -> caseSSub role

caseS -> DPs pS 
caseS -> Name pS 
caseSSub -> DPSubs pS
caseSSub -> Name pS

DP -> det role caseO 
DPSub -> detSub role caseO

caseO -> of DP 
caseO -> of Name

DP -> det commonNoun
DPs -> det commonNounMod
DPs -> det commonNounHead
DPSub -> detSubcommonNoun
DPSubs -> detSubcommonNounModHead

detSubcommonNoun -> detSub commonNounMod
detSubcommonNoun -> detSub commonNounHead
detSubcommonNounModHead -> detSub commonNounModHead

commonNounModHead -> commonNounMod
commonNounModHead -> commonNounHead

Name -> Name andName 
Name -> Namex andName 
Namex -> Name punctName 
Namex -> Namex punctName 
andName -> conjN Name 
andName -> punctN conjN Name 
punctName -> punctN Name

commonNoun -> commonNoun andCommonNoun
commonNoun -> commonNounx andCommonNoun
commonNounx -> commonNoun punctCommonNoun
commonNounx -> commonNounx punctCommonNoun
andCommonNoun -> conjN commonNoun 
andCommonNoun -> punctN conjN commonNoun 
punctCommonNoun -> punctN commonNoun

role -> role androle
role -> rolex androle 
rolex -> role punctrole 
rolex -> rolex punctrole 
androle -> conjN role 
androle -> punctN conjN role
punctrole -> punctN role


commonNoun -> F commonNounHead 
commonNounHead -> F commonNounHead 
role -> F roleHead
role -> Cnt of nat
commonNoun -> P commonNounHead
commonNounHead -> P commonNounHead

commonNoun -> AdjNa commonNounHead
commonNoun -> AdjNa commonNounMod
commonNounMod -> AdjNa commonNounHead
commonNounMod -> AdjNa commonNounMod
role -> AdjNa roleHead
role -> AdjNa role







punct -> ","
punctN -> ","

Cnt -> 'country'
nat -> 'nationality'
P -> 'production'
F -> 'film' | 'art' | 'executive' | 'costume'

V -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VT -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
Vsr -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VsrT -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VMod -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VsrMod -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VPassOrgT -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VPassOrgMod -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VOrgPassT -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VOrgPassMod -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'




Name -> 'M0' | 'M1' | 'M2' | 'M3' | 'M4' | 'M5' | 'M6' | 'M7' | 'M8' | 'M9'

commonNoun -> 'character' | 'person' | 'composer' | 'prequel' | 'director' | 'writer' | 'company' | 'actor' | 'designer' | 'founder' | 'sequel' | 'producer' | 'spouse' | 'child' | 'editor' | 'employer' | 'employee' | 'distributor' | 'sibling' | 'star' | 'parent' | 'cinematographer' | 'screenwriter' | 'film'
commonNounHead -> 'character' | 'person' | 'composer' | 'prequel' | 'director' | 'writer' | 'company' | 'actor' | 'designer' | 'founder' | 'sequel' | 'producer' | 'spouse' | 'child' | 'editor' | 'employer' | 'employee' | 'distributor' | 'sibling' | 'star' | 'parent' | 'cinematographer' | 'screenwriter' | 'film'

role -> 'character' | 'person' | 'composer' | 'prequel' | 'director' | 'writer' | 'company' | 'actor' | 'designer' | 'founder' | 'sequel' | 'producer' | 'spouse' | 'child' | 'editor' | 'employer' | 'employee' | 'distributor' | 'sibling' | 'star' | 'parent' | 'cinematographer' | 'screenwriter' | 'film'
roleHead -> 'character' | 'person' | 'composer' | 'prequel' | 'director' | 'writer' | 'company' | 'actor' | 'designer' | 'founder' | 'sequel' | 'producer' | 'spouse' | 'child' | 'editor' | 'employer' | 'employee' | 'distributor' | 'sibling' | 'star' | 'parent' | 'cinematographer' | 'screenwriter' | 'film'


NPQ -> 'who' | 'what' | 'Who' | 'What'
WhW -> 'What' | 'Which' | 'what' | 'which'
did -> 'did' | 'Did'

conj -> 'and'
conjN -> 'and'

pS -> "'s"
of -> 'of'

det -> 'a' | 'an'
detSub -> 'a' | 'an'

by -> 'by'
Adj -> 'female' | 'American' | 'French' | 'Italian' | 'male' | 'Swedish' | 'Canadian' | 'British' | 'Spanish' | 'Mexican' | 'Chinese' | 'German' | 'American' | 'Dutch' | 'Japanese'
AdjNa -> 'female' | 'American' | 'French' | 'Italian' | 'male' | 'Swedish' | 'Canadian' | 'British' | 'Spanish' | 'Mexican' | 'Chinese' | 'German' | 'American' | 'Dutch' | 'Japanese'
was -> 'was' | 'were' | 'Was' | 'Were'
R -> 'that'
whose -> 'whose'
Qmark -> '?'






"""

In [32]:
src_to_target_grammar = {
    "S -> NPQ VP":"S -> NPQ が VP か", 
    "S -> NPQ was NominalSub":"S -> NominalSub は NPQ でしたか", # Q
    "S -> NPQ did NPVMod":"S -> NPVMod のは NPQ でしたか", # what did A write and B edit? -> A ga kaite, B ga hennshushita no ha nann deshitaka
    "S -> was NominalSub Vobl":"S -> NominalSub は Vobl か",
    "S -> NPQ Vobl":"S -> NPQ が Vobl か", # same Q
    "S -> was NominalSub Adj":"S -> NominalSub は Adj でしたか",
    "S -> was NominalSub Nominal":"S -> NominalSub は Nominal でしたか",
    "S -> did NominalSub VP":"S -> NominalSub は VP か",

    # TODO: Chain rules to add for VMod and VPrepOrgMod
    "NPVMod -> Nominal VMod":"NPVMod -> Nominal が VMod",
    # todo: this rule basically follows VP, VPrepOrg finally derives as V
    "NPVMod -> Nominal VPrepOrgMod":"NPVMod -> Nominal を VPrepOrgMod", # VPrepOrg: passive in EN, active in JP 
    "NPVT -> Nominal VT":"NPVT -> Nominal が VT",
    "NPVT -> Nominal VPrepOrgT":"NPVT -> Nominal を VPrepOrgT",

    "VP -> V Nominal":"VP -> Nominal を V", # have lunch -> hirugohan wo taberu
    "VP -> was Vobl":"VP -> Vobl", # was done by him -> kare ni sareta
    "VPT -> VT Nominal":"VPT -> Nominal を VT",
    "VPT -> was VoblT":"VPT -> VoblT",
    "VPMod -> VMod Nominal":"VPMod -> Nominal を VMod",
    "VPMod -> was VoblMod":"VPMod -> VoblMod",

    "VPrep -> was VPrep":"VPrep -> VPrep",
    "VPrep -> Vsr by":"VPrep -> Vsr",
    "VPrepT -> was VPrepT":"VPrepT -> VPrepT",
    "VPrepT -> VsrT by":"VPrepT -> VsrT",
    "VPrepMod -> was VPrepMod":"VPrepMod -> VPrepMod",
    "VPrepMod -> VsrMod by":"VPrepMod -> VsrMod",
 
    "VPrepOrgMod -> was VPrepOrgMod":"VPrepOrgMod -> VPrepOrgMod",
    "VPrepOrgMod -> VPassOrgMod by":"VPrepOrgMod -> VPassOrgMod",
    "VPrepOrgT -> was VPrepOrgT":"VPrepOrgT -> VPrepOrgT",
    "VPrepOrgT -> VPassOrgT by":"VPrepOrgT -> VPassOrgT",

    "Vobl -> VPrep Nominal":"Vobl -> Nominal に VPrep", # done by nominal -> nominal ni sareta
    "VoblT -> VPrepT Nominal":"VoblT -> Nominal に VPrepT",
    "VoblMod -> VPrepMod Nominal":"VoblMod -> Nominal に VPrepMod",
    
    # "NPQ -> WhW Nominal":"NPQ -> WhW Nominal", # which city -> dono machi
    # TODO: caseO -> of ... : caseO ... no
    "NPQ -> WhW role caseO":"NPQ -> caseO WhW role", # which character of Alice -> alisu no dono kyarakuta

    "commonNoun -> commonNoun RC":"commonNoun -> RC commonNoun", # people whose role was... -> yaku ga... hito
    "WhWcommonNoun -> WhWcommonNoun RC":"WhWcommonNoun -> RC WhWcommonNoun",
    "detSubcommonNoun -> detSubcommonNoun RC":"detSubcommonNoun -> RC detSubcommonNoun",

    # "RC -> VoblMod":"RC -> VoblMod", # people attacked by... -> ...ni kougekisareta hito 
    "RC -> R VPMod":"RC -> VPMod", # people that eat... -> ...wo taberu hito
    "RC -> R NPVMod":"RC -> NPVMod", # people that you like -> anata ga sukina hito
    "RC -> whose role VPMod":"RC -> role が VPMod", # todo: role GA -> ...

    # "VP -> VPT andVP":"VP -> VPT andVP",
    # "VP -> VPx andVP":"VP -> VPx andVP",
    # "VPx -> VPT punctVPT":"VPx -> VPT punctVPT",
    # "VPx -> VPx punctVPT":"VPx -> VPx punctVPT",
    # "andVP -> conj VP":"andVP -> conj VP",
    "andVP -> punct conj VP":"andVP -> conj VP",
    # "punctVPT -> punct VPT":"punctVPT -> punct VPT",

    # VPMod -> VPT andVPMod
    # VPMod -> VPx andVPMod
    # VPx -> VPT punctVPT 
    # VPx -> VPx punctVPT 
    # andVPMod -> conj VPMod
    "andVPMod -> punct conj VPMod":"andVPMod -> conj VPMod",
    # punctVPT -> punct VPT

    # Vobl -> VoblT andVobl
    # Vobl -> Voblx andVobl 
    # Voblx -> VoblT punctVoblT 
    # Voblx -> Voblx punctVoblT 
    # andVobl -> conj Vobl 
    "andVobl -> punct conj Vobl":"andVobl -> conj Vobl",
    # punctVoblT -> punct VoblT

    # VoblMod -> VoblT andVoblMod
    # VoblMod -> Voblx andVoblMod 
    # Voblx -> VoblT punctVoblT 
    # Voblx -> Voblx punctVoblT 
    # andVoblMod -> conj VoblMod 
    "andVoblMod -> punct conj VoblMod":"andVoblMod -> conj VoblMod",
    # punctVoblT -> punct VoblT

    # VPrep -> VPrepT andVPrep 
    # VPrep -> VPrepX andVPrep 
    # VPrepX -> VPrepT punctVPrepT 
    # VPrepX -> VPrepX punctVPrepT 
    # andVPrep -> conj VPrep 
    "andVPrep -> punct conj VPrep":"andVPrep -> conj VPrep",
    # punctVPrepT -> punct VPrepT

    # VPrepMod -> VPrepT andVPrepMod 
    # VPrepMod -> VPrepX andVPrepMod 
    # VPrepX -> VPrepT punctVPrepT 
    # VPrepX -> VPrepX punctVPrepT 
    # andVPrepMod -> conj VPrepMod 
    "andVPrepMod -> punct conj VPrepMod":"andVPrepMod -> conj VPrepMod",
    # punctVPrepT -> punct VPrepT

    # VPrepOrgMod -> VPrepOrgT andVPrepOrgMod 
    # VPrepOrgMod -> VPrepOrgX andVPrepOrgMod 
    # VPrepOrgX -> VPrepOrgT punctVPrepOrgT 
    # VPrepOrgX -> VPrepOrgX punctVPrepOrgT 
    # andVPrepOrgMod -> conj VPrepOrgMod 
    "andVPrepOrgMod -> punct conj VPrepOrgMod":"andVPrepOrgMod -> conj VPrepOrgMod", 
    # punctVPrepOrgT -> punct VPrepOrgT

    # V -> VT andV
    # V -> Vx andV
    # Vx -> VT punctVT
    # Vx -> Vx punctVT 
    # andV -> conj V
    "andV -> punct conj V":"andV -> conj V", 
    # punctVT -> punct VT

    # VT -> VT andVT
    # VT -> Vx andVT
    # Vx -> VT punctVT
    # Vx -> Vx punctVT 
    # andVT -> conj VT
    "andVT -> punct conj VT":"andVT -> conj VT", 
    # punctVT -> punct VT

    # VMod -> VT andVMod
    # VMod -> Vx andVMod
    # andVMod -> conj VMod
    "andVMod -> punct conj VMod":"andVMod -> conj VMod",

    # Vsr -> VsrT andVsr
    # Vsr -> Vsrx andVsr
    # Vsrx -> VsrT punctVsrT
    # Vsrx -> Vsrx punctVsrT 
    # andVsr -> conj Vsr
    "andVsr -> punct conj Vsr":"andVsr -> conj Vsr",
    # punctVsrT -> punct VsrT

    # VsrT -> VsrT andVsrT
    # VsrT -> Vsrx andVsrT
    # andVsrT -> conj VsrT
    "andVsrT -> punct conj VsrT":"andVsrT -> conj VsrT", 
    # punctVsrT -> punct VsrT

    # VsrMod -> VsrT andVsrMod
    # VsrMod -> Vsrx andVsrMod
    # andVsrMod -> conj VsrMod
    "andVsrMod -> punct conj VsrMod":"andVsrMod -> conj VsrMod",

    # Vx -> Vx punctVPrepT
    # Vx -> VT punctVPrepT
    # V -> Vx andVPrep
    # V -> VT andVPrep
    # VPrep -> VPrepT andV
    # VPrep -> VPrepX andV 
    # VPrepX -> VPrepT punctVT 
    # VPrepX -> VPrepX punctVT


    # VPrepOrgMod -> VPrepOrgT andVOrgPassMod
    # VPrepOrgMod -> VPrepOrgX andVOrgPassMod
    # VPrepOrgX -> VPrepOrgT punctVOrgPassT
    # VPrepOrgX -> VPrepOrgX punctVOrgPassT

    # punctVOrgPassT -> punct VOrgPassT
    # andVOrgPassMod -> conj VOrgPassMod
    "andVOrgPassMod -> punct conj VOrgPassMod":"andVOrgPassMod -> conj VOrgPassMod",


    # VPassOrgMod -> VPassOrgT andVPassOrgMod
    # VPassOrgMod -> VPassOrgx andVPassOrgMod
    # VPassOrgx -> VPassOrgT punctVPassOrgT
    # VPassOrgx -> VPassOrgx punctVPassOrgT 
    # andVPassOrgMod -> conj VPassOrgMod
    "andVPassOrgMod -> punct conj VPassOrgMod":"andVPassOrgMod -> conj VPassOrgMod", 
    # punctVPassOrgT -> punct VPassOrgT


    # VMod -> Vx andVPrepMod
    # VMod -> VT andVPrepMod
    # VPrepMod -> VPrepT andVMod
    # VPrepMod -> VPrepX andVMod

    # NPVMod -> NPVT andNPVMod
    # NPVMod -> NPVx andNPVMod 
    # NPVx -> NPVT punctNPVT
    # NPVx -> NPVx punctNPVT 
    # andNPVMod -> conj NPVMod 
    "andNPVMod -> punct conj NPVMod":"andNPVMod -> conj NPVMod", 
    # punctNPVT -> punct NPVT

    # V -> F V
    # VT -> F VT
    # VMod -> F VMod
    # Vsr -> F Vsr
    # VsrT -> F VsrT
    # VsrMod -> F VsrMod
    # VPassOrgMod -> F VPassOrgMod
    # VPassOrgT -> F VPassOrgT
    # VOrgPassT -> F VOrgPassT
    # VOrgPassMod -> F VOrgPassMod


    # Nominal -> Name 
    # Nominal -> DP
    # Nominal -> commonNoun

    # NominalSub -> Name
    # NominalSub -> DPSub
    # NominalSub -> commonNoun


    "DP -> det role caseO":"DP -> caseO role",
    "DPSub -> detSub role caseO":"DPSub -> caseO detSub role",
# of -> no
    "caseO -> of DP":"caseO -> DP of",
    "caseO -> of Name":"caseO -> Name of",

    # DP -> det commonNoun
    # DPSub -> detSub commonNoun

    # Name -> Name andName 
    # Name -> Namex andName 
    # Namex -> Name punctName 
    # Namex -> Namex punctName 
    # andName -> conjN Name 
    "andName -> punctN conjN Name":"andName -> conjN Name",
    # punctName -> punctN Name

    # commonNoun -> commonNoun andCommonNoun
    # commonNoun -> commonNounx andCommonNoun
    # commonNounx -> commonNoun punctCommonNoun
    # commonNounx -> commonNounx punctCommonNoun
    # andCommonNoun -> conjN commonNoun 
    "andCommonNoun -> punctN conjN commonNoun":"andCommonNoun -> conjN commonNoun",
    # punctCommonNoun -> punctN commonNoun

    # role -> role androle
    # role -> rolex androle 
    # rolex -> role punctrole 
    # rolex -> rolex punctrole 
    # androle -> conjN role 
    "androle -> punctN conjN role":"androle -> conjN role",
    # punctrole -> punctN role

    # commonNoun -> F commonNoun 
    # role -> F role
    # role -> Cnt of nat
    # commonNoun -> P commonNoun

    "role -> Cnt of nat":"role -> nat Cnt" # country of nationality -> 国籍国
    
    # commonNoun -> AdjNa commonNoun 
    # role -> AdjNa role
}

special cases: like -> ga suki

In [33]:
pattern_pairs = {'を結婚し':'と結婚し','を影響し':'に影響し'}

def post_processing(sens, pattern_pairs = pattern_pairs):
    new_sens = []
    for sen in sens:
        for org, tar in pattern_pairs.items():
            sen = sen.replace(org, tar)
        new_sens.append(sen)
    return new_sens

In [34]:
word_parser = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dict())

In [6]:
with open('Questions.txt') as f:
    lines = f.readlines()

lines = list(set(lines))
del lines[lines.index('\n')]

## Word Parsing

In [43]:
tag_word_set, failed_sentences, ambiguity_sentences = word_parser.parse_words(sentences=lines)

100%|██████████| 105461/105461 [16:48<00:00, 104.53it/s]


Word parsing completed! 0 sentences failed. 37299 sentences occurred ambiguity.


In [44]:
len(ambiguity_sentences)

37299

In [6]:
# reparsing
tag_word_set_2, failed_sentences_2, ambiguity_sentences_2 = word_parser.parse_words(sentences=['Was a Chinese actor that M2 was written by and starred a composer\n'])

Word parsing completed! 0 sentences failed. 1 sentences occurred ambiguity.


In [15]:
tag_word_set.keys()

dict_keys(['did', 'Name', 'V', 'det', 'role', 'of', 'commonNoun', 'pS', 'AdjNa', 'F', 'VT', 'conjN', 'conj', 'punct', 'R', 'VMod', 'P', 'punctN', 'VsrT', 'by', 'VsrMod', 'was', 'whose', 'VPassOrgMod', 'Vsr', 'VPassOrgT', 'VOrgPassMod', 'Cnt', 'nat', 'VOrgPassT', 'detSub', 'Adj', 'WhW', 'NPQ'])

In [78]:
import nltk
analyze_sen = "Who was influenced by a composer influenced by M3 and influenced by M4 and M5 and influenced by M1"
for i in ambiguity_sentences[analyze_sen]:
    i.draw()

NameError: name 'ambiguity_sentences' is not defined

## Translation

In [55]:
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)

In [31]:
trans_sentence, trans_map = e2j_translator.translate(ambiguity_sentences, remove_space=True)

100%|██████████| 19823/19823 [05:08<00:00, 64.32it/s]


In [36]:
len(trans_map)

784

In [37]:
trans_map

[{'M3とM5はM1に影響されてそしてM2に影響されてそしてM4に編集された撮影監督に監督されましたか': 10,
  'M3とM5はM1に影響されてそしてM2に影響された撮影監督に監督されてそしてM4に編集されましたか': 10,
  'M3とM5はM1に影響された撮影監督に監督されてそしてM2に影響されてそしてM4に編集されましたか': 10},
 {'あるスウェーデンの映画はM1に雇用されたプロデューサでしたか': 4, 'あるスウェーデンの映画プロデューサはM1に雇用されましたか': 3},
 {'M1とM2が結婚したのはどの雇用主がM3の作家と撮影監督を雇用した映画プロデューサでしたか': 5,
  'M1とM2が結婚したのはどの雇用主がM3の作家を雇用した映画プロデューサと撮影監督でしたか': 5},
 {'ある映画はM1、M2、M3、とM4に影響された編集者でしたか': 4, 'ある映画編集者はM1、M2、M3、とM4に影響されましたか': 3},
 {'ある映画はM3の前日譚の監督に影響されてそしてM1とM2に影響されたプロデューサでしたか': 6,
  'ある映画プロデューサはM3の前日譚の監督に影響されてそしてM1とM2に影響されましたか': 5},
 {'ある映画はM1、M2、M3、M4、M5、とM6に雇用されてそしてM7に雇用された監督でしたか': 6,
  'ある映画監督はM1、M2、M3、M4、M5、とM6に雇用されてそしてM7に雇用されましたか': 5},
 {'ある映画はM1に雇用された監督でしたか': 4, 'ある映画監督はM1に雇用されましたか': 3},
 {'ある映画はM1、M2、とM3に雇用されてそしてM4、M5、とM6に雇用された監督でしたか': 6,
  'ある映画監督はM1、M2、とM3に雇用されてそしてM4、M5、とM6に雇用されましたか': 5},
 {'何がM2を書いたスウェーデンの人にプロデュースされてそしてM0を主演させてそして監督されましたか': 11,
  '何がM2を書いてそして主演したスウェーデンの人にプロデュースされてそしてM0に監督されましたか': 11,
  '何がM2を書いたスウェーデンの人にプロデュースされてそしてM0に主演してそして監督されましたか': 11},
 {'何がM5の創設者を

#### Disambiguity
784 out of 19823 ambiguities harmful

In [79]:
# with open('ambiguinous_question.txt', 'w') as f:
#     for line in ambiguity_sentences:
#         f.write(line)
try: 
    ambiguity_sentences
except NameError:
    ambiguity_sentences = None


if ambiguity_sentences == None:
    with open('ambiguinous_question.txt', 'r') as f:
        ambiguity_sentences = f.readlines()


In [9]:
pref_pattern = [{"S -> was NominalSub Vobl","commonNoun -> F commonNounHead"},
                {"S -> was NominalSub Vobl","commonNounHead -> F commonNounHead"},
                {"caseS -> Name pS"},
                {"NPQ -> WhWNominal"}
                ]

In [87]:
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)
trans_sentence, trans_map = e2j_translator.translate(ambiguity_sentences, remove_space=True, prefered_pattern=pref_pattern)

100%|██████████| 37299/37299 [08:02<00:00, 77.35it/s] 


In [89]:
trans_map

{"Was a prequel of M0 edited by M2 's editor and star and executive produced by M1\n": {'M0のある前日譚はM2の編集者と主演に編集され、M1にエグゼクティブプロデュースされましたか': 7,
  'M0のある前日譚はM2の編集者に編集され、M1に主演し、エグゼクティブプロデュースされましたか': 7},
 'Did M3 , M4 , M5 , and M6 edit a film whose writer was employed by M1 and founded M2\n': {'M3、M4、M5、とM6は作家がM1に雇用された映画を編集し、M2を創設しましたか': 8,
  'M3、M4、M5、とM6は作家がM1に雇用され、M2を創設した映画を編集しましたか': 8},
 'Did M3 star a screenwriter that directed M1 and M2 and star M4 and M5\n': {'M3はM1とM2を監督した脚本家を主演させ、M4とM5を主演されましたか': 6,
  'M3はM1とM2を監督し、M4とM5を主演させた脚本家を主演されましたか': 6},
 'What film director influenced by M1 and influenced by a person that founded M4 and M5 produced M2\n': {'M4とM5を創設したM1に影響され、人に影響されたどの映画監督がM2をプロデュースしましたか': 10,
  'M1に影響され、M4とM5を創設した人に影響されたどの映画監督がM2をプロデュースしましたか': 10},
 'Did M3 marry a cinematographer that married M2 and influenced M1\n': {'M3はM2を結婚した撮影監督を結婚し、M1を影響しましたか': 6,
  'M3はM2を結婚し、M1を影響した撮影監督を結婚しましたか': 6},
 'What film was edited by a cinematographer that directed M5 and was written by M1

536 | 438 | 388 | 377 out of 17648

In [79]:
test_sent =  'Who was influenced by a composer influenced by M3 and influenced by M4 and M5 and influenced by M1'
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)
trans_sentence, trans_map = e2j_translator.translate(test_sent, remove_space=True, prefered_pattern=pref_pattern)

100%|██████████| 1/1 [00:00<00:00, 18.91it/s]


In [80]:
trans_map

{'Who was influenced by a composer influenced by M3 and influenced by M4 and M5 and influenced by M1': {'誰がM3に影響され、M4とM5に影響され、M1に影響された作曲家に影響されましたか': 11,
  '誰がM3に影響され、M4とM5に影響された作曲家に影響され、M1に影響されましたか': 11,
  '誰がM3に影響された作曲家に影響され、M4とM5に影響され、M1に影響されましたか': 11}}

In [81]:
trans_sentence

['誰がM3に影響され、M4とM5に影響され、M1に影響された作曲家に影響されましたか']

In [82]:
import nltk 
analyze_sen = test_sent
trees = word_parser.parser.parse(analyze_sen.split())
ts = [t for t in trees]
for i in ts:
    i.draw()

## Test/Evaluation

In [8]:
import random
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
vq = random.choices(lines, k=1000)

In [71]:
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)
trans_vq, trans_map_vq = e2j_translator.translate(vq, remove_space=True, prefered_pattern=pref_pattern)

100%|██████████| 1000/1000 [00:10<00:00, 91.12it/s]


In [66]:
vq_trans = [(s,t) for s, t in zip(vq, trans_vq)]

### Evaluate with GOLD

In [51]:
gd_jp = pd.read_csv('CWQ Annotation - GOLD-JP.csv')

In [52]:
dev, test = train_test_split(gd_jp, test_size=0.66, random_state=42)

In [53]:
scr_dev = list(dev['questionPatternModEntities'])
scr_test = list(test['questionPatternModEntities'])
gd_dev = list(dev['questionPatternModEntitiesJP'])
gd_test = list(test['questionPatternModEntitiesJP'])

In [54]:
pref_pattern = [{"S -> was NominalSub Vobl","commonNoun -> F commonNounHead"},
                {"S -> was NominalSub Vobl","commonNounHead -> F commonNounHead"},
                {"caseS -> Name pS"},
                {"NPQ -> WhWNominal"}
                ]
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)
trans_dev, trans_map_dev = e2j_translator.translate(scr_dev, remove_space=True, prefered_pattern=pref_pattern)
trans_dev = post_processing(trans_dev,pattern_pairs=pattern_pairs)

100%|██████████| 52/52 [00:01<00:00, 51.40it/s]


In [55]:
from sacrebleu.metrics import BLEU
from sacrebleu.tokenizers import tokenizer_ja_mecab

In [56]:
bleu = BLEU(tokenize='ja-mecab')
bleu.corpus_score(hypotheses=trans_dev, references=[gd_dev])

BLEU = 97.98 99.3/98.4/97.5/96.9 (BP = 1.000 ratio = 1.000 hyp_len = 1206 ref_len = 1206)

On dev: BLEU = 97.98 99.3/98.4/97.5/96.9 (BP = 1.000 ratio = 1.000 hyp_len = 1206 ref_len = 1206)

In [57]:
comp = []
for a, b in zip(gd_dev,trans_dev):
    if a != b:
        comp.append((a, b))

In [58]:
len(comp)

6

In [59]:
comp

[('M0はM1のどの配給者とプロデューサーを創設しましたか', 'M0が創設したのはM1のどの配給者とプロデューサーでしたか'),
 ('M1とM2に影響し、M4とM5と結婚したある俳優はM3の創設者でしたか',
  'M1とM2に影響し、M4とM5が結婚したある俳優はM3の創設者でしたか'),
 ('M0が買収したのは会社のどの子会社と親会社でしたか', 'M0が買収したのは会社のどの子と親でしたか'),
 ('ある衣裳デザイナーはM3のカナダ人の男性の配偶者でしたか', 'ある衣裳デザイナーはM3のカナダの男性の配偶者でしたか'),
 ('M4のアメリカ人の撮影監督はM0、M1とM2を編集し、監督し、書きましたか',
  'M4のアメリカの撮影監督はM0、M1とM2を編集し、監督し、書きましたか'),
 ('M3、M4、M5とM6に監督され、プロデュースされ、書かれたどの映画がM1とM2に主演されましたか',
  'M3、M4、M5とM6に監督され、プロデュースされ、書かれたどの映画がM1とM2を主演させましたか')]

In [60]:
pref_pattern = [{"S -> was NominalSub Vobl","commonNoun -> F commonNounHead"},
                {"S -> was NominalSub Vobl","commonNounHead -> F commonNounHead"},
                {"caseS -> Name pS"},
                {"NPQ -> WhWNominal"}
                ]
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)
trans_test, trans_map_test = e2j_translator.translate(scr_test, remove_space=True, prefered_pattern=pref_pattern)
trans_test = post_processing(trans_test,pattern_pairs=pattern_pairs)

100%|██████████| 103/103 [00:01<00:00, 75.56it/s]


In [61]:
bleu = BLEU(tokenize='ja-mecab')
bleu.corpus_score(hypotheses=trans_test, references=[gd_test])

BLEU = 97.07 99.2/97.9/96.4/95.0 (BP = 1.000 ratio = 1.000 hyp_len = 2306 ref_len = 2307)

On test: BLEU = 97.07 99.2/97.9/96.4/95.0 (BP = 1.000 ratio = 1.000 hyp_len = 2306 ref_len = 2307)

In [46]:
comp_test = []
for a, b in zip(gd_test,trans_test):
    if a != b:
        comp_test.append((a, b))

In [47]:
len(comp_test)

19

In [48]:
comp_test

[('M0が創設したのはどの映画監督の雇用主でしたか', 'M0が創設したのは映画監督のどの雇用主でしたか'),
 ('M4とM5のある主演、編集者と撮影監督はM0、M1とM2のスペイン人の作家と監督でしたか',
  'M4とM5のある主演、編集者と撮影監督はM0、M1とM2のスペインの作家と監督でしたか'),
 ('M0、M1とM2は何を監督し、編集し、書きましたか', 'M0、M1とM2が監督し、編集し、書いたのは何でしたか'),
 ('M1のどの続編はM0を主演させましたか', 'M1のどの続編がM0を主演させましたか'),
 ('何がM0にエグゼクティブ・プロデュースされ、M1に書かれましたか', '何がM0にエグゼクティブプロデュースされ、M1に書かれましたか'),
 ('どの映画プロデューサーがM1とM2の編集者、エグゼクティブプロデューサー、監督と作家でしたか',
  'M1とM2のある編集者、エグゼクティブプロデューサー、監督と作家はどの映画プロデューサーでしたか'),
 ('M1、M2、M3とM4の編集者と主演は誰でしたか', 'M1、M2、M3とM4のある編集者と主演は誰でしたか'),
 ('ある衣裳デザイナーはM2のスウェーデン人の配偶者でしたか', 'ある衣裳デザイナーはM2のスウェーデンの配偶者でしたか'),
 ('M1のどの雇用主はM0の親会社でしたか', 'M0の親はM1のどの雇用主でしたか'),
 ('M6のどの配給者とプロデューサーはM0とM1に買収され、M2、M3、M4とM5に買収されましたか',
  'M6のどの配給者とプロデューサーがM0とM1に買収され、M2、M3、M4とM5に買収されましたか'),
 ('ある衣裳デザイナーはM2のスウェーデン人の親でしたか', 'ある衣裳デザイナーはM2のスウェーデンの親でしたか'),
 ('M1、M2とM3はどの映画を書き、編集し、プロデュースし、監督しましたか',
  'M1、M2とM3が書き、編集し、プロデュースし、監督したのはどの映画でしたか'),
 ('M6のどの配給者とプロデューサーはM0とM1に買収され、M2、M3、M4とM5を買収しましたか',
  'M6のどの配給者とプロデューサーがM0とM1に買収され、M2、M3、M4とM5を買収しましたか'),
 ('M0、M1、M2

# Dataset translation

In [75]:
# load questions
with open('Questions.txt') as f:
    lines = f.readlines()

lines = list(set(lines))
del lines[lines.index('\n')]

In [76]:
pref_pattern = [{"S -> was NominalSub Vobl","commonNoun -> F commonNounHead"},
                {"S -> was NominalSub Vobl","commonNounHead -> F commonNounHead"},
                {"caseS -> Name pS"},
                {"NPQ -> WhWNominal"}
                ]
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)
lines_jp, maps_jp = e2j_translator.translate(lines, remove_space=True, prefered_pattern=pref_pattern)
lines_jp = post_processing(lines_jp,pattern_pairs=pattern_pairs)

100%|██████████| 105461/105461 [19:51<00:00, 88.53it/s] 


In [64]:
lines_jp

['あるカナダの映画プロデューサーはM2を監督しましたか',
 '誰がM1に影響され、M2に影響され、M3の男性の兄弟姉妹に影響されましたか',
 '誰がM1のプロデューサーの創設者に影響され、影響しましたか',
 'どの人がM4と結婚し、M1に影響され、M2に影響され、M3に影響されましたか',
 'M1は俳優の配偶者にエグゼクティブプロデュースされましたか',
 'ある映画の作家と撮影監督はM3を監督し、M0、M1とM2をエグゼクティブプロデュースしましたか',
 'M1は撮影監督の親の兄弟姉妹でしたか',
 'M2は人にエグゼクティブプロデュースされ、M1の撮影監督に書かれ、M3、M4とM5に監督されましたか',
 'M1は作曲家に書かれ、プロデュースされましたか',
 'M2を書いたあるフランスの男性の俳優は作曲家でしたか',
 'M2を監督し、書き、主演したある撮影監督はM0でしたか',
 '何がM0に書かれ、M1にプロデュースされ、撮影監督のスウェーデンの配偶者にプロデュースされましたか',
 'M3とM4に影響されたある俳優はM1とM2に雇用されましたか',
 'M0のスウェーデンの創設者はM2、M3とM4を監督しましたか',
 'M1はM0の主演の親でしたか',
 '何がM1が主演した映画プロデューサーに監督され、エグゼクティブプロデュースされましたか',
 '映画プロデューサーのある親はM0、M1、M2とM3をエグゼクティブプロデュースしましたか',
 '何がM1の続編の編集者に書かれ、M0にエグゼクティブプロデュースされましたか',
 '映画のある監督、エグゼクティブプロデューサー、編集者と撮影監督はM1でしたか',
 'M0の撮影監督、作家と作曲家がプロデュースしたのは何でしたか',
 'M1はM0の編集者に編集され、プロデュースされ、監督され、書かれ、エグゼクティブプロデュースされましたか',
 'どの映画がM1に編集され、フランスの人に編集されましたか',
 '会社が買収し、M0が買収し、M1が買収したのは何でしたか',
 'M2のオランダの親は映画監督でしたか',
 'M1が影響し、M2、M3とM4が影響し、M7が結婚し、M5とM6が影響したのは誰でしたか',
 '何が人の雇用主に買収され、M0に買収されましたか',
 '何がM1に書かれ

In [67]:
len(lines) == len(lines_jp)
lines_en2jp = {}
for en, jp in zip(lines, lines_jp):
    lines_en2jp[en] = jp

In [70]:
# import csv

# with open('ambiguity_maps.csv', 'w') as f:
#     for key in maps_jp.keys():
#         f.write("%s, %s\n" % (key, maps_jp[key]))